In [13]:
import pandas as pd

# Load each relevant sheet
sheet12_df = pd.read_excel('TravelTrendsToUK.xlsx', sheet_name='12', skiprows=2)
sheet13_df = pd.read_excel('TravelTrendsToUK.xlsx', sheet_name='13', skiprows=2)


In [14]:
print("Column names in sheet 12:", sheet12_df.columns.tolist())

try:
    VisitsByCountry_df = sheet12_df.melt(id_vars=['Area of Residence', 'Country'], var_name='Year', value_name='Visits by Country')
    print(VisitsByCountry_df.tail())
    print("Data extraction successful.")
except KeyError as e:
    print(f"Column error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

Column names in sheet 12: ['Area of Residence', 'Country', '2019', '2020', '2021', '2022', '2023', '2019 Q1', '2019 Q2', '2019 Q3', '2019 Q4', '2020 Q1', '2020 Q2', '2020 Q3', '2020 Q4', '2021 Q1', '2021 Q2', '2021 Q3', '2021 Q4', '2022 Q1', '2022 Q2', '2022 Q3', '2022 Q4', '2023 Q1', '2023 Q2', '2023 Q3', '2023 Q4']
     Area of Residence                       Country     Year  \
1595   Other Countries                        Mexico  2023 Q4   
1596   Other Countries  Other Central & Sth. America  2023 Q4   
1597   Other Countries             Rest of the World  2023 Q4   
1598   Other Countries         Total Other Countries  2023 Q4   
1599       Total World                   Total World  2023 Q4   

     Visits by Country  
1595             49000  
1596             86000  
1597                 0  
1598           1688000  
1599           9441000  
Data extraction successful.


In [15]:
print("Column names in sheet 13:", sheet13_df.columns.tolist())

try:
    SpendingByCountry_df = sheet13_df.melt(id_vars=['Area of Residence', 'Country'], var_name='Year', value_name='Spendings by Country')
    print(SpendingByCountry_df.tail())
    print("Data extraction successful.")
except KeyError as e:
    print(f"Column error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

Column names in sheet 13: ['Area of Residence', 'Country', '2019', '2020', '2021', '2022', '2023', '2019 Q1', '2019 Q2', '2019 Q3', '2019 Q4', '2020 Q1', '2020 Q2', '2020 Q3', '2020 Q4', '2021 Q1', '2021 Q2', '2021 Q3', '2021 Q4', '2022 Q1', '2022 Q2', '2022 Q3', '2022 Q4', '2023 Q1', '2023 Q2', '2023 Q3', '2023 Q4']
     Area of Residence                       Country     Year  \
1595   Other Countries                        Mexico  2023 Q4   
1596   Other Countries  Other Central & Sth. America  2023 Q4   
1597   Other Countries             Rest of the World  2023 Q4   
1598   Other Countries         Total Other Countries  2023 Q4   
1599       Total World                   Total World  2023 Q4   

     Spendings by Country  
1595                   35  
1596                   93  
1597                    0  
1598                 2489  
1599                 7428  
Data extraction successful.


In [16]:
Country_df = pd.merge(VisitsByCountry_df, SpendingByCountry_df, on=['Area of Residence', 'Country', 'Year'], how='inner')
Country_df.tail()

,Area of Residence,Country,Year,Visits by Country,Spendings by Country
1595,Other Countries,Mexico,2023 Q4,49000,35
1596,Other Countries,Other Central & Sth. America,2023 Q4,86000,93
1597,Other Countries,Rest of the World,2023 Q4,0,0
1598,Other Countries,Total Other Countries,2023 Q4,1688000,2489
1599,Total World,Total World,2023 Q4,9441000,7428


In [17]:
import numpy as np
# Ensure the columns 'Period' and 'Year' are treated as strings
Country_df['Year'] = Country_df['Year'].astype(str)


# Replace "Not available" and other placeholders with NaN for easier processing
Country_df.replace("Not available", pd.NA, inplace=True)

print(Country_df[Country_df['Year'] == '2020'][['Area of Residence','Country','Year','Visits by Country','Spendings by Country']])

# Save the updated data to a new CSV file
output_path = 'CountryBasedUpdated_TravelTrends.csv'
Country_df.to_csv(output_path, index=False)


    Area of Residence                       Country  Year Visits by Country  \
64      North America                        Canada  2020              <NA>   
65      North America                           USA  2020              <NA>   
66      North America           Total North America  2020           1171000   
67             Europe                       Austria  2020              <NA>   
68             Europe                       Belgium  2020              <NA>   
..                ...                           ...   ...               ...   
123   Other Countries                        Mexico  2020              <NA>   
124   Other Countries  Other Central & Sth. America  2020              <NA>   
125   Other Countries             Rest of the World  2020              <NA>   
126   Other Countries         Total Other Countries  2020           1942000   
127       Total World                   Total World  2020          11101000   

    Spendings by Country  
64                  <NA>

In [18]:
# Ensure the 'Year' column is treated as a string
Country_df['Year'] = Country_df['Year'].astype(str)

def backward_fill_values(data, year_to_fill, quarters, column_to_fill):
    # Loop through each area of residence
    for area in data['Area of Residence'].unique():
        # Loop through each country within the area
        for country in data[data['Area of Residence'] == area]['Country'].unique():
            print(f"Processing Area of Residence: {area}, Country: {country}")

            for quarter in quarters:
                # Create the period to fill
                period_to_fill = f'{year_to_fill} {quarter}'
                next_year = str(int(year_to_fill) + 1)
                next_period = f'{next_year} {quarter}'

                # Get the value from the next period
                next_value = data.loc[
                    (data['Year'] == next_period) & 
                    (data['Area of Residence'] == area) & 
                    (data['Country'] == country), 
                    column_to_fill
                ].values

                # Ensure there's a value to backfill from the next period
                if len(next_value) > 0 and not pd.isna(next_value[0]):
                    data.loc[
                        (data['Year'] == period_to_fill) & 
                        (data['Area of Residence'] == area) & 
                        (data['Country'] == country), 
                        column_to_fill
                    ] = next_value[0]

                    print(f"Filled {period_to_fill} with {next_period} value: {next_value[0]}")

    return data

# Define the quarters and columns to fill
quarters_to_impute = ['Q1', 'Q2', 'Q3', 'Q4']
columns_to_impute = ['Visits by Country', 'Spendings by Country']

# Step 1: Apply the backward fill function for 2020
for column in columns_to_impute:
    Country_df = backward_fill_values(Country_df, '2020', quarters_to_impute, column)

# Step 2: Recalculate the yearly data for 2020 based on the filled quarters
for area in Country_df['Area of Residence'].unique():
    for country in Country_df[Country_df['Area of Residence'] == area]['Country'].unique():
        for column in columns_to_impute:
            # Filter the data for 2020 and the relevant quarters
            yearly_value = Country_df.loc[
                (Country_df['Year'].str.startswith('2020')) & 
                (Country_df['Area of Residence'] == area) & 
                (Country_df['Country'] == country) & 
                (Country_df['Year'].str.contains('Q')), 
                column
            ].sum()
            
            # Update the yearly row with the sum of quarters
            Country_df.loc[
                (Country_df['Year'] == '2020') & 
                (Country_df['Area of Residence'] == area) & 
                (Country_df['Country'] == country), 
                column
            ] = yearly_value

            print(f"Recalculated 2020 for {country} in {area}: {yearly_value}")

# Print the entire DataFrame after recalculating yearly data for 2020
print("After Recalculating - Entire DataFrame for 2020:")
print(Country_df[Country_df['Year'].str.startswith('2020')])

# Save the updated data to a new CSV file
output_path = 'CountryBasedUpdated_TravelTrends.csv'
Country_df.to_csv(output_path, index=False)

Processing Area of Residence: North America, Country: Canada
Filled 2020 Q1 with 2021 Q1 value: 2000
Filled 2020 Q2 with 2021 Q2 value: 4000
Filled 2020 Q3 with 2021 Q3 value: 50000
Filled 2020 Q4 with 2021 Q4 value: 72000
Processing Area of Residence: North America, Country: USA
Filled 2020 Q1 with 2021 Q1 value: 29000
Filled 2020 Q2 with 2021 Q2 value: 47000
Filled 2020 Q3 with 2021 Q3 value: 239000
Filled 2020 Q4 with 2021 Q4 value: 348000
Processing Area of Residence: North America, Country: Total North America
Filled 2020 Q1 with 2021 Q1 value: 31000
Filled 2020 Q2 with 2021 Q2 value: 51000
Filled 2020 Q3 with 2021 Q3 value: 289000
Filled 2020 Q4 with 2021 Q4 value: 421000
Processing Area of Residence: Europe, Country: Austria
Filled 2020 Q1 with 2021 Q1 value: 3000
Filled 2020 Q2 with 2021 Q2 value: 4000
Filled 2020 Q3 with 2021 Q3 value: 24000
Filled 2020 Q4 with 2021 Q4 value: 25000
Processing Area of Residence: Europe, Country: Belgium
Filled 2020 Q1 with 2021 Q1 value: 2000
F